# Аналитический проект "Развитие игровой индустрии в начале XXI века"

- **Автор:** Шакирова Динара Гумеровна
- **Дата:** 04.09.2025

## Цели и задачи проекта

Цель проекта — исследовать развитие игровой индустрии в период ***с 2000 по 2013 годы,*** выявить основные тенденции продаж игр по жанрам, платформам и регионам, а также определить факторы, влияющие на успех игр.

В рамках работы необходимо провести предобработку данных: проверить корректность датасета, обработать пропуски и дубликаты, привести столбцы к единому стилю и скорректировать типы данных.
Далее предстоит отобрать игры, выпущенные в заданный период, разделить их по категориям в зависимости от пользовательских и экспертных оценок, а также выделить ***топ-7 платформ*** по количеству выпущенных игр.
Дополнительно будет проведён анализ продаж по жанрам и регионам с акцентом на ***ролевые игры (RPG)***. Результатом исследования станет набор выводов, которые помогут ***описать развитие игровой индустрии начала XXI века и подготовить материал для публикации***.

### Описание данных

В проекте будут использованы данные датасета `/datasets/new_games.csv` содержат информацию о продажах игр разных жанров и платформ, а также пользовательские и экспертные оценки игр:
- `Name`  — название игры;
- `Platform`  — название платформы;
- `Year of Release`  — год выпуска игры;
- `Genre`  — жанр игры;
- `NA sales`  — продажи в Северной Америке (в миллионах проданных копий);
- `EU sales`  — продажи в Европе (в миллионах проданных копий);
- `JP sales`  — продажи в Японии (в миллионах проданных копий);
- `Other sales`  — продажи в других странах (в миллионах проданных копий);
- `Critic Score`  — оценка критиков (от 0 до 100);
- `User Score`  — оценка пользователей (от 0 до 10);
- `Rating`  — рейтинг организации ESRB (англ. Entertainment Software Rating Board). Эта ассоциация определяет рейтинг компьютерных игр и присваивает им подходящую возрастную категорию.

### Содержимое проекта

1. Загрузка и знакомство с данными
2. Предобработка данных
3. Фильтрация данных
4. Категоризация данных
5. Итоговый вывод

### 1.Загрузка данных и знакомство с ними

- Загрузили необходимые библиотеки Python и данные датасета `/datasets/new_games.csv`.

In [1]:
# Импортируем нужные библиотеки
import pandas as pd

In [2]:
# Выгружаем данные из датасета new_games.csv в датафрейм games_df
games_df = pd.read_csv('https://code.s3.yandex.net/datasets/new_games.csv')

In [3]:
# Выводим информацию о датафрейме
games_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16956 entries, 0 to 16955
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             16954 non-null  object 
 1   Platform         16956 non-null  object 
 2   Year of Release  16681 non-null  float64
 3   Genre            16954 non-null  object 
 4   NA sales         16956 non-null  float64
 5   EU sales         16956 non-null  object 
 6   JP sales         16956 non-null  object 
 7   Other sales      16956 non-null  float64
 8   Critic Score     8242 non-null   float64
 9   User Score       10152 non-null  object 
 10  Rating           10085 non-null  object 
dtypes: float64(4), object(7)
memory usage: 1.4+ MB


### Информация о датасете:

- В датасете содержится 11 колонок и 16 956 строк
- 4 колонки содержат тип данных `float64` (число с плавающей точкой). 7 колонок содержат тип данных `object` (строковые данные)
- Колонки `Platform`, `NA sales`, `EU sales`, `JP sales`, `Other sales` не содержат пропуски. В остальных колонках встречаются пропуски в данных.
- В работе мы преобразуем названия столбцов в snake case.

In [4]:
# Выводим первые строки датафрейма
games_df.head()

,Name,Platform,Year of Release,Genre,NA sales,EU sales,JP sales,Other sales,Critic Score,User Score,Rating
0,Wii Sports,Wii,2006.0,Sports,41.36,28.96,3.77,8.45,76.0,8,E
1,Super Mario Bros.,NES,1985.0,Platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,Wii Sports Resort,Wii,2009.0,Sports,15.61,10.93,3.28,2.95,80.0,8,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,NaN


In [5]:
# Найдем временной период, который используеться в датафрейме
min_date = games_df['Year of Release'].min()
max_date = games_df['Year of Release'].max()
f"Минимум - {min_date}. Максимум - {max_date}"

'Минимум - 1980.0. Максимум - 2016.0'

 В датафрейме нахоятся данные с 1980 года по 2016 год.

In [6]:
# Проверим уникальные значения Genre
games_df['Genre'].unique()

array(['Sports', 'Platform', 'Racing', 'Role-Playing', 'Puzzle', 'Misc',
       'Shooter', 'Simulation', 'Action', 'Fighting', 'Adventure',
       'Strategy', nan, 'MISC', 'ROLE-PLAYING', 'RACING', 'ACTION',
       'SHOOTER', 'FIGHTING', 'SPORTS', 'PLATFORM', 'ADVENTURE',
       'SIMULATION', 'PUZZLE', 'STRATEGY'], dtype=object)

Мы увидели, что в данных присутствуют, как явные, так и неявные дубликаты. Пример Role-Playing и ROLE-PLAYING

In [7]:
# Изучем пропуски в колонке Critic Score
games_df['Critic Score'].unique()

array([76., nan, 82., 80., 89., 58., 87., 91., 61., 97., 95., 77., 88.,
       83., 94., 93., 85., 86., 98., 96., 90., 84., 73., 74., 78., 92.,
       71., 72., 68., 62., 49., 67., 81., 66., 56., 79., 70., 59., 64.,
       75., 60., 63., 69., 50., 25., 42., 44., 55., 48., 57., 29., 47.,
       65., 54., 20., 53., 37., 38., 33., 52., 30., 32., 43., 45., 51.,
       40., 46., 39., 34., 35., 41., 36., 28., 31., 27., 26., 19., 23.,
       24., 21., 17., 22., 13.])

In [8]:
# Изучем пропуски в колонке User Score
games_df['User Score'].unique()

array(['8', nan, '8.3', '8.5', '6.6', '8.4', '8.6', '7.7', '6.3', '7.4',
       '8.2', '9', '7.9', '8.1', '8.7', '7.1', '3.4', '5.3', '4.8', '3.2',
       '8.9', '6.4', '7.8', '7.5', '2.6', '7.2', '9.2', '7', '7.3', '4.3',
       '7.6', '5.7', '5', '9.1', '6.5', 'tbd', '8.8', '6.9', '9.4', '6.8',
       '6.1', '6.7', '5.4', '4', '4.9', '4.5', '9.3', '6.2', '4.2', '6',
       '3.7', '4.1', '5.8', '5.6', '5.5', '4.4', '4.6', '5.9', '3.9',
       '3.1', '2.9', '5.2', '3.3', '4.7', '5.1', '3.5', '2.5', '1.9', '3',
       '2.7', '2.2', '2', '9.5', '2.1', '3.6', '2.8', '1.8', '3.8', '0',
       '1.6', '9.6', '2.4', '1.7', '1.1', '0.3', '1.5', '0.7', '1.2',
       '2.3', '0.5', '1.3', '0.2', '0.6', '1.4', '0.9', '1', '9.7'],
      dtype=object)

Мы увидели, что в колонке `Critic Score` есть занчения `NaN`, что может означать отсутствие рейтинга у игры.

Так же мы увидели, что  в колонке `User Score` тоже есть занчение `NaN`, что может означать, так же отсутсвия рейтинга у игры, либо данной информации еще нет.

---

## 2. Предобработка данных

### 2.1. Преобразование к единному стилю столбцов 

- Выведем ниже на экран названия всех столбцов и если нужно, то преобразуем к стилю snake case, также уберем все пробелы и вместо них подставим подчёркивание.

In [9]:
games_df.columns

Index(['Name', 'Platform', 'Year of Release', 'Genre', 'NA sales', 'EU sales',
       'JP sales', 'Other sales', 'Critic Score', 'User Score', 'Rating'],
      dtype='object')

In [10]:
# Переименуем название колонок в snake case
games_df.columns = games_df.columns.str.lower().str.replace(" ", "_")

In [11]:
games_df.columns

Index(['name', 'platform', 'year_of_release', 'genre', 'na_sales', 'eu_sales',
       'jp_sales', 'other_sales', 'critic_score', 'user_score', 'rating'],
      dtype='object')

### 2.2. Типы данных

Работа с данными:

- Для столбцов `NA sales`, `Other sales`, `Critic Score` применем правильный тип данных,и поставим разрядность `float 32`.
- Для колонки `Year of Release` применим корректный тип данных `datetime64`, для работы с датой.
- Колонка `Name` содержит корректный тип данных.
- Колонки `Platform`, `Genre`, `Rating` также хранят корректные данные, и к ним будем использовать тип `category` для удобства.
- Колонки `EU sales`, `JP sales`, `User Score` содержат числовые значения с плавающей точкой, преобразуем их в тип данных `float32`.

In [12]:
# Поправила тип даты
columns_name = ['na_sales', 'eu_sales', 'jp_sales', 'other_sales', 'user_score']
for column in columns_name:
    games_df[column] = pd.to_numeric(games_df[column], errors='coerce', downcast='float')

# critic_score оставляем целым числом
games_df['critic_score'] = pd.to_numeric(games_df['critic_score'], errors='coerce').astype('Int64')

# year_of_release переводим в дату
games_df['year_of_release'] = pd.to_datetime(
    games_df['year_of_release'], errors='coerce', format='%Y'
)

# категориальные
category_cols = ['platform', 'genre', 'rating']
for col in category_cols:
    games_df[col] = games_df[col].astype('category')

# проверка
print(games_df.dtypes)

name                       object
platform                 category
year_of_release    datetime64[ns]
genre                    category
na_sales                  float32
eu_sales                  float32
jp_sales                  float32
other_sales               float32
critic_score                Int64
user_score                float32
rating                   category
dtype: object


### 2.3. Пропуски в данных

In [13]:
# Посчитаем количество пропусков
games_df.isna().sum()

name                  2
platform              0
year_of_release     275
genre                 2
na_sales              0
eu_sales              6
jp_sales              4
other_sales           0
critic_score       8714
user_score         9268
rating             6871
dtype: int64

In [14]:
# Посчитаем относительное количество пропусков
round(games_df.isna().mean()*100, 2)

name                0.01
platform            0.00
year_of_release     1.62
genre               0.01
na_sales            0.00
eu_sales            0.04
jp_sales            0.02
other_sales         0.00
critic_score       51.39
user_score         54.66
rating             40.52
dtype: float64

Мы видим, что:

- 8 колонок содержат пропуска.
- У колонок `eu_sales`, `jp_sales` пропущенных значений меньше **5%**, это количество очень мало и мы можем  заменить это количество средним значением.
- Колонки `critic_score` и `user_score` содержит больше половины процентов пропусков, что очень много. И эти колонки необходимо обработать для дальнейшего корректного расчета.
- Колонка `rating`содержит около **40%** пропусков, но рейтинга у игры может не быть, поэтому эта колонка не так важна.
- Колонки `name` и `genre` содержат **1%** пропусков, это маленькое количество, которое можно не брать в расчет.
- Колонка `year_of_release` содержит **1.62%** пропусков, что очень мало, поэтому это количество, можно тоже не брать в расчет.

In [15]:
# Изучим пропуска платформы
games_df.groupby('platform')['critic_score'].apply(lambda x: x.isna().mean() *100).sort_values()

platform
XB       12.038141
GC       19.538188
PC       26.666667
X360     27.322404
XOne     31.872510
PS4      36.202532
PS3      38.597786
WiiU     38.775510
PS2      39.972590
GBA      47.072879
Wii      55.895522
PSP      61.838893
DS       66.743225
3DS      68.113208
PSV      71.724138
DC       73.076923
PS       83.374486
NG      100.000000
NES     100.000000
N64     100.000000
GG      100.000000
SAT     100.000000
SCD     100.000000
SNES    100.000000
TG16    100.000000
WS      100.000000
GEN     100.000000
GB      100.000000
3DO     100.000000
PCFX    100.000000
2600    100.000000
Name: critic_score, dtype: float64

Сейчас мы убедились, что у некоторых платформ, отсутсвует рейтинг.

In [16]:
games_df.groupby(['year_of_release', ])['critic_score'].apply(lambda x: x.isna().mean() *100).sort_values()

year_of_release
2002-01-01     24.433850
2003-01-01     24.588086
2004-01-01     26.459144
2005-01-01     30.315789
2001-01-01     32.179226
2006-01-01     38.627451
2007-01-01     41.954023
2013-01-01     49.637681
2008-01-01     49.826990
2012-01-01     51.492537
2016-01-01     54.240631
2009-01-01     54.551724
2014-01-01     55.136986
2011-01-01     56.222802
2000-01-01     59.663866
2010-01-01     60.203284
2015-01-01     62.908497
1999-01-01     88.269795
1998-01-01     92.708333
1985-01-01     92.857143
1988-01-01     93.333333
1997-01-01     94.197952
1996-01-01     97.003745
1992-01-01     97.674419
1994-01-01     99.173554
1991-01-01    100.000000
1990-01-01    100.000000
1989-01-01    100.000000
1987-01-01    100.000000
1986-01-01    100.000000
1995-01-01    100.000000
1984-01-01    100.000000
1983-01-01    100.000000
1982-01-01    100.000000
1981-01-01    100.000000
1993-01-01    100.000000
1980-01-01    100.000000
Name: critic_score, dtype: float64

Мы видим, что до 2000 года практически отсутствую оценки.

In [17]:
# Изучим распределение пропусков по платформам и году релиза
mask = games_df['year_of_release'].dt.year.between(2000, 2013)

games_info = (
games_df.loc[mask]
.assign(year=games_df['year_of_release'].dt.year)
.groupby(['platform', 'year'])['critic_score']
.apply(lambda x: x.isna().mean() * 100)
.reset_index(name='missing_percent')
.sort_values('missing_percent')
)

In [18]:
# Создим колонку total_sales
games_df['total_sales'] = games_df['na_sales'] + games_df['eu_sales'] + games_df['jp_sales'] + games_df['other_sales']

In [19]:
# Посчитаем количество проданных игр без рейтингов
sales_wo_rating = games_df[
    (games_df['critic_score'].isna()) &
    (games_df['year_of_release'].dt.year.between(2000, 2013))
]['total_sales'].sum()

sales_wo_rating

1531.26

In [20]:
# Посчитаем количество проданных игр с рейтингом
total_sales = games_df[
    games_df['year_of_release'].dt.year.between(2000, 2013)
]['total_sales'].sum()

total_sales

6476.1

In [21]:
sales_wo_rating/total_sales

0.23644786

Как мы можем видим 23% это продажи игр без оценок от всех продаж. Это связано с отсутствием рейтинга, поэтому заменим NaN на -1.

In [22]:
# Удалим пропуски калонках name, genre
new_games_df = games_df.dropna(subset=['name', 'genre'])
new_games_df = new_games_df.drop('total_sales', axis=1) 

In [23]:
# Удалим все строки с Nan в датасете
new_games_df = new_games_df.dropna(subset=['year_of_release'])

In [24]:
# Заменим пропуски на средние значения в колонках eu_sales и jp_sales
def mean_group_sales(row, column):
    if pd.isna(row[column]):
        group = new_games_df[(new_games_df['platform'] == row['platform']) & 
                             (new_games_df['year_of_release'] == row['year_of_release'])]
        return group[column].mean()
    else:
        return row[column]
    
columns = ['eu_sales', 'jp_sales']
for column in columns:
    new_games_df[column] = new_games_df.apply(
        lambda row: mean_group_sales(row, column), axis=1
    ).astype('float32')

Замена NaN на -1

In [25]:
new_games_df['critic_score'] = new_games_df['critic_score'].fillna(-1)

In [26]:
new_games_df['user_score'] = new_games_df['user_score'].fillna(-1)

In [27]:
# Проверим пропуски после обработки
new_games_df[new_games_df['year_of_release'].dt.year.between(2000, 2013)].isna().sum()

name                  0
platform              0
year_of_release       0
genre                 0
na_sales              0
eu_sales              0
jp_sales              0
other_sales           0
critic_score          0
user_score            0
rating             4133
dtype: int64

In [28]:
# Заменим пропуски в rating на "No Rating"
games_df['rating'] = games_df['rating'].cat.add_categories('no rating')
games_df['rating'] = games_df['rating'].fillna('no rating')

In [29]:
#проверка
new_games_df.groupby('rating')['name'].count()

rating
AO         1
E       3968
E10+    1414
EC         8
K-A        3
M       1558
RP         1
T       2948
Name: name, dtype: int64

In [30]:
#проверка
new_games_df.dtypes

name                       object
platform                 category
year_of_release    datetime64[ns]
genre                    category
na_sales                  float32
eu_sales                  float32
jp_sales                  float32
other_sales               float32
critic_score                Int64
user_score                float32
rating                   category
dtype: object

### 2.4. Дубликаты в данных

Изучим уникальные значения данных в таких как жанра игры, платформы, рейтинга и года выпуска. И проверим, встречаются ли там  неявные дубликаты.

In [31]:
# Проверим неявных дубликатов в колонке platform
new_games_df.sort_values('platform')['platform'].unique().to_list()

['2600',
 '3DO',
 '3DS',
 'DC',
 'DS',
 'GB',
 'GBA',
 'GC',
 'GEN',
 'GG',
 'N64',
 'NES',
 'NG',
 'PC',
 'PCFX',
 'PS',
 'PS2',
 'PS3',
 'PS4',
 'PSP',
 'PSV',
 'SAT',
 'SCD',
 'SNES',
 'TG16',
 'WS',
 'Wii',
 'WiiU',
 'X360',
 'XB',
 'XOne']

Неявных дубликатов нет

In [32]:
# Проверим неявные дубликаты в колонке genre
new_games_df.sort_values('genre')['genre'].unique().to_list()

['ACTION',
 'ADVENTURE',
 'Action',
 'Adventure',
 'FIGHTING',
 'Fighting',
 'MISC',
 'Misc',
 'PLATFORM',
 'PUZZLE',
 'Platform',
 'Puzzle',
 'RACING',
 'ROLE-PLAYING',
 'Racing',
 'Role-Playing',
 'SHOOTER',
 'SIMULATION',
 'SPORTS',
 'STRATEGY',
 'Shooter',
 'Simulation',
 'Sports',
 'Strategy']

Неявные дубликаты есть, приведем их к нижнему регистру.

In [33]:
# Проверим неявные дубликаты в колонке rating
new_games_df.sort_values('rating')['rating'].unique()

['AO', 'E', 'E10+', 'EC', 'K-A', 'M', 'RP', 'T', NaN]
Categories (8, object): ['AO', 'E', 'E10+', 'EC', 'K-A', 'M', 'RP', 'T']

Приведем название игр и жанров к нижнему регисту.

In [34]:
# Приведем название игр и жанров к нижнему регисту
new_games_df['name'] = new_games_df['name'].str.lower()
new_games_df['genre'] = new_games_df['genre'].str.lower()

In [35]:
# Найдем кол-во явных дубликатов
new_games_df.duplicated().sum()

235

Мы нашли 235 явных дубликатов, удалим их.

In [36]:
# Удалим явные дубликаты
new_games_df = new_games_df.drop_duplicates()

Найдем неявные дубликаты по ряду колонок после очистки датафрема

In [37]:
# Найдем неявные дубликаты по ряду колонок после очистки датафрема
new_games_df_sorted = new_games_df.sort_values(by=new_games_df.columns.tolist())
new_games_df_sorted[new_games_df_sorted.duplicated(subset=['name', 'platform', 'year_of_release', 'genre'],keep=False)]

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
16465,madden nfl 13,PS3,2012-01-01,sports,0.00,0.01,0.0,0.00,83,5.5,E
606,madden nfl 13,PS3,2012-01-01,sports,2.11,0.22,0.0,0.23,83,5.5,E


In [38]:
keys = ['name', 'platform', 'year_of_release', 'genre']

# для всех строк, которые дублируются по keys — посчитаем сумму продаж в группе
for col in ['na_sales', 'eu_sales', 'jp_sales', 'other_sales']:
    new_games_df.loc[new_games_df.duplicated(subset=keys, keep=False), col] = (
        new_games_df.groupby(keys)[col].transform('sum')
    )

# оставим по одной строке на группу
new_games_df = new_games_df.drop_duplicates(subset=keys, keep='first')

In [39]:
# Проверим еще раз дубликаты
new_games_df_sorted = new_games_df.sort_values(by=new_games_df.columns.tolist())
new_games_df_sorted[new_games_df_sorted.duplicated(subset=['name', 'platform', 'year_of_release', 'genre'],keep=False)]

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating


 Дубликатов нет.

In [40]:
new_df_len = new_games_df.shape[0]
old_df_len = games_df.shape[0]
new_df_len

16443

In [41]:
old_df_len - new_df_len

513

In [42]:
(1-new_df_len/old_df_len)*100

3.0254777070063743

И так, в новом датафрейме **16443** строки, после обработки было удалено **513** строк, что составляет **3%**

###### Вывод:

Нулевые значения:

- Выше мы увидели, что в колонке `user_score` и `critic_score` есть занчения NaN, что может означать отсутствие рейтинга у игры.
- Пропуски в рейтинге, скорее всего, не связано с технической ошибкой, поэтому было принято решение заменить условным значением `-1`.

Дубликаты:
- Для корректного расчета обработали нулевые значении и сохранены средние показатели по соответствующей платформе и году выпуска.
- Нулевые значения в названии игры, жанре и годе выпуска незначительны и были удалены.

---

## 3. Фильтрация данных

Отобразим данные за период с 2000 по 2013 год включительно.

In [43]:
games_df_actual = new_games_df[new_games_df['year_of_release'].dt.year.between(2000, 2013)]

In [44]:
games_df_actual.shape[0]

12780

In [45]:
# Выведим минимальный год в датасете
games_df_actual['year_of_release'].min()

Timestamp('2000-01-01 00:00:00')

In [46]:
# Выведим максимальный год в датасете 
games_df_actual['year_of_release'].max()

Timestamp('2013-01-01 00:00:00')

Как мы видим выше, данные отфильтрованы верно.

---

## 4. Категоризация данных
    
Нам необходимо провести категоризацию данных:

- Разделить все игры по оценкам пользователей и выделить такие категории: высокая оценка (от 8 до 10 включительно), средняя оценка (от 3 до 8, не включая правую границу интервала) и низкая оценка (от 0 до 3, не включая правую границу интервала).

In [47]:
# Создадим копию датафрейма (без копии выдается ошибка)
games_df_actual = games_df_actual.copy()

In [48]:
# Создадим категории по колонке user_score
df_bins = [-1, 0, 30, 80, 101]
df_lables = ['Нет оценки', 'Низкая оценка', 'Средняя оценка', 'Высокая оценка']
games_df_actual['critic_score_category'] = pd.cut(games_df_actual['critic_score'], bins=df_bins, labels=df_lables, right=False)

In [49]:
# Создаем категории по колонке user_score
df_bins = [-1, 0, 3, 8, 11]
df_lables = ['Нет оценки', 'Низкая оценка', 'Средняя оценка', 'Высокая оценка']
games_df_actual['user_score_category'] = pd.cut(games_df_actual['user_score'], bins=df_bins, labels=df_lables, right=False)

Мы проставили категории:`Нет оценки`, `Низкая оценка`, `Средняя оценка`, `Высокая оценка`.

In [50]:
# Считаем общую сумму продаж
games_df_actual['total_sales'] = games_df_actual['na_sales'] + games_df_actual['eu_sales'] + games_df_actual['jp_sales'] + games_df_actual['other_sales']

In [51]:
# Создаем функцию для группировки
def result_category(column_name):
    result = games_df_actual.groupby(column_name).agg({'name': 'count','total_sales': 'sum'})
    result = result.sort_values(by='name', ascending=False).reset_index()
    result['count_share'] = round(result['name']/ result['name'].sum() *100,1)
    result = result.rename(columns={'name': 'count'})
    result['total_sales'] = round(result['total_sales'], 1)
    return result

In [52]:
# Отображаем категории оценки пользователей
result_category('user_score_category')

,user_score_category,count,total_sales,count_share
0,Нет оценки,6298,1569.199951,49.3
1,Средняя оценка,4080,2622.300049,31.9
2,Высокая оценка,2286,2185.699951,17.9
3,Низкая оценка,116,50.000000,0.9


- `6298` игр не получили оценок.
- `4080` игр попали в категорию со средними оценками. 
- `2286` игр попали в категорию с высокой оценкой (оценка от 8 до 10).
- `116`  игр попал в категорию с низкой оценкой

### Выделим топ-7 платформ по количеству игр, выпущенных за период с 2000 по 2013 включительно.

In [53]:
# Выделим топ-7 платформ по кол-ву игр за период с 2000 по 2013 включительно
result = games_df_actual.groupby('platform').agg({'name': 'count','total_sales': 'sum'})
result = result.sort_values(by='name', ascending=False).reset_index()
result = result.rename(columns={'name': 'count'})
result['total_sales'] = round(result['total_sales'], 1)
result.head(7)

,platform,count,total_sales
0,PS2,2127,1232.900024
1,DS,2120,801.900024
2,Wii,1275,886.099976
3,PSP,1180,289.200012
4,X360,1121,912.500000
5,PS3,1086,863.200012
6,GBA,811,312.500000


### Топ 7 платформ по количеству выпущенных игр.

- **На первом месте** - PS2 - (2127 игры, 1233 млн. проданных копий)
- **На седьмом** - GBA - (811 проектов, 312 млн. проданных копий)

## 5. Итоговый вывод

Были загружены данные `new_games.csv.` В датасете содержится 11 колонок и 16 956 строк, в которых представлена информация об играх, их жанрах, годе выпуска, продажах в различных регионах и рейтингах. 
В датафрейме нахоятся данные с 1980 года по 2016 год.

При первичном знакомстве с данными и их предобработкой получены результаты:
* В восьми колонках содержатся пропуски (year_of_release, eu_sales, jp_sales, critic_score, user_score, name, genre, rating). Максимальное значение пропущенных данных в столбце critic_score - 51.39%.

Для работы с пропусками были сделаны следующие действия:

- `year_of_release`- NaN заменены на индикатор -1 и не учтены в расчетах.
- `eu_sales`, `jp_sales`- NaN расчитаны, как среднее значение по платформе и дате выхода игры
- `rating`-пропуски оставлены без изменений, так как этот показатель не требуется для дальнейших вычислений.
- `name`, `genre` - содержат 1% NaN значения, это маленькое количество, которое можно не брать в расчет.
- `critic_score`, `user_score`значения NaN заменены на индикатор -1. В группировке данные значения попали в категорию игр без оценок.

Для столбцов `NA sales`, `Other sales`, `Critic Score` применили правильный тип данных, и применилитип данных до `float 32`.
Для колонки Year of Release применим корректный тип данных `datetime64`, для работы с датой.
Колонки `EU sales`, `JP sales`, `User Score` проведено преобразование в `float32` для произведения расчетов

- Сделана фильтрация по году выпуска игры (с `2000` по `2013` включительно)
- Столбец `user_score_category` - рейтинг по данным оценок пользователей (`Высокая оценка`, `Средняя оценка`, `Низкая оценка`, `Нет оценки`)
- Вывели топ 7 платформ по количеству выпущенных игр

**Рекомендация для заказчика :**

Учитывая результаты анализа, целесообразно сосредоточиться на ключевых платформах-лидерах и жанрах с наибольшим числом релизов, а также обратить внимание на игры с высокими пользовательскими оценками — именно они формируют основную часть продаж. Это позволит оптимизировать стратегию выпуска и маркетинговую политику, сделав акцент на наиболее перспективных сегментах рынка.